In [40]:
#hide
#default_exp update_project
from nbdev.showdoc import show_doc

In [41]:
#export
import os
import pkg_resources
import subprocess

import jinja2
from fastcore.script import *

from kicad_helpers import get_git_root, get_project_metadata, get_gitignore_list, in_gitignore
from kicad_helpers.core import _set_root

# update_project

> Functions scripts to initialize a new kicad-helpers project or to update an existing project.

* toc: true

Similar to `nbdev`: [nbdev_new](https://nbdev.fast.ai/tutorial.html#Set-up-Repo)

* [x] github workflows
* [x] .kicad_helpers_config
* [x] kitspace.yaml
* [ ] Example notebooks (e.g., for syncing the BOM)

Templates are stored in the "kicad_helpers/templates" folder, and are included in the python package by adding the following line to the `MANIFEST.in` file:

```
graft kicad_helpers/templates
```

In [42]:
#export
@call_parse
def update_templates(v:Param("verbose", bool),
                     overwrite:Param("overwrite existing templates", bool),
                     root:Param("project root directory", str)="."):
    """
    Update project templates from the `kicad_helpers/templates` directory
    (ignoring anything in the project's `.gitignore` list).
    """
    templates_path = os.path.abspath(pkg_resources.resource_filename('kicad_helpers', 'templates'))
    root = _set_root(root)
    metadata = get_project_metadata(root)
    
    file_list = []
    for root_, dirs, files in os.walk(templates_path):
        if len(files):
            for file in files:
                path = os.path.join(root_[len(templates_path) + 1:], file)
                if not in_gitignore(path):
                    src_path = os.path.abspath(os.path.join(templates_path, path))
                    dst_path = os.path.abspath(os.path.join(root, path))
                    
                    # Create the `dst_path` directory if it doesn't exist
                    os.makedirs(os.path.split(dst_path)[0], exist_ok=True)

                    if os.path.exists(dst_path):
                        if v:
                            print(f"{ path } already exists")
                        if not overwrite:
                            continue
    
                    with open(src_path) as f:
                        template = jinja2.Template(f.read())

                    with open(dst_path, "w") as f:
                        if v:
                            print(f"Rendering { path } template.")
                        f.write(template.render(**metadata))

In [43]:
#hide
from kicad_helpers import setup_test_repo

root = os.path.join(get_git_root("."), "_temp")
setup_test_repo(root)

```sh
kh_update --help
```

In [44]:
#hide_input
print(subprocess.check_output(f"kh_update --help", shell=True).decode("utf-8"))

usage: kh_update [-h] [--v] [--overwrite] [--root ROOT]

Update project templates from the `kicad_helpers/templates` directory (ignoring anything in the project's `.gitignore`
list).

optional arguments:
  -h, --help   show this help message and exit
  --v          verbose (default: False)
  --overwrite  overwrite existing templates (default: False)
  --root ROOT  project root directory (default: .)



```sh
kh_update --v --overwrite
```

In [45]:
#hide_input
print(subprocess.check_output(f"kh_update --v --overwrite --root { root }", shell=True).decode("utf-8"))

kitspace.yaml already exists
Rendering kitspace.yaml template.
.github/workflows/build.yml already exists
Rendering .github/workflows/build.yml template.
.kicad_helpers_config/config.kibot.yaml already exists
Rendering .kicad_helpers_config/config.kibot.yaml template.
.kicad_helpers_config/manufacturers/PCBWay.kibot.yaml already exists
Rendering .kicad_helpers_config/manufacturers/PCBWay.kibot.yaml template.
tests/Tests.ipynb already exists
Rendering tests/Tests.ipynb template.



# Project structure

Here's an example showing the recommended KiCad project structure.

In [46]:
#hide input
print(subprocess.check_output(f"cd { root } && tree --dirsfirst -d -I $(python -c \"from kicad_helpers import get_gitignore_list; print('kicad_helpers|' + get_gitignore_list())\") .", shell=True).decode("utf-8"))

.
├── docs
│   ├── png
│   ├── src
│   └── svg
├── library
├── manufacturing
│   └── default
│       ├── assembly_photos
│       ├── gerbers
│       └── position
└── tests

11 directories

